# Librerie utilizzate

In [17]:
from pulp import * # per modellare il problema e risolverlo
import numpy as np # per creare array
import math #per arrotondamenti superiori
import csv

# Master problem o problema ristretto

In [18]:
class MasterProblem:
	def __init__(self, lunghezza_max, lunghezza_assi, domanda_assi, patterns, mostraRisultati):
		
		self.lunghezza_max=lunghezza_max
		self.lunghezze_assi=lunghezza_assi
		self.domande_assi=domanda_assi
		self.patterns= patterns
		
		self.prob = LpProblem('MasterProblem',LpMinimize)
		self.obj = LpConstraintVar("obj")
		self.prob.setObjective(self.obj)

		self.iterazione = 0
		self.mostraRisultati = mostraRisultati
		
		# generazione lista di vincoli
		self.PatternVars=[]
		self.lista_vincoli=[]
		for index,value in enumerate(domanda_assi):
			var=LpConstraintVar("C"+str(index),LpConstraintGE,value)  # generazione vincolo a_ij * x_j >= d_i
			self.lista_vincoli.append(var)
			self.prob+=var

		# generazione della matrice A quindi i valori a_ij
		for i,x in enumerate(self.patterns):
			temp=[]
			for j,y in enumerate(x):
				# solo quelle >0 per creare la variabile
				if y>0: 
					temp.append(j)

			# generazione di variabili con solo coeffiecienti > 0, cioè quanti pezzi di un certo tipo per il pattern i-esimo
			var=LpVariable("x"+str(i + 1), 0, None, LpContinuous, lpSum(self.obj+[self.lista_vincoli[v] for v in temp]))
			self.PatternVars.append(var)
		
	# risolvi il problem master restituendo la soluzione duale u*
	def risolvi(self):
		self.prob.writeLP('master.lp')
		self.prob.solve()
		if self.mostraRisultati:
			print("Iterazione", iterazione)
			print("Ottimo primale", value(self.prob.objective))
		self.iterazione = self.iterazione + 1
		return [self.prob.constraints[i].pi for i in self.prob.constraints]
		
			
	# column generation => aggiungi il pattern al problema ristretto
	def aggiungiPattern(self,pattern):
		
		self.patterns.append(pattern)
		temp=[]
		
		for j,y in enumerate(pattern):
			if y>0: 
				temp.append(j)

		# aggiungi le variabili con lo stesso criterio di prima
		var=LpVariable("x"+str(len(self.patterns) + 1)	, 0, None, LpContinuous, lpSum(self.obj+[pattern[v]*self.lista_vincoli[v] for v in temp]))
		self.PatternVars.append(var)

	def getOttimo(self):
		return math.ceil(value(self.prob.objective))
	
	def getPatternsSoluzione(self):
		patternList=[]
		for i,x in enumerate(self.PatternVars):
			if value(x)>0:
				patternList.append((value(x),self.patterns[i]))
		return patternList

	#arrotonda la soluzione a interi quando non ci sono nuovi pattern (soluzione ottima)
	def arrotonda(self):
		for var in self.prob.variables():
				var.cat = LpInteger
			

# Slave problem

In [19]:
class SlaveProblem:
    def __init__(self,duale, itemLengths,maxValue):
        self.prob=LpProblem("SlaveProblem",LpMaximize)
        self.lista_var=[LpVariable('q'+str(i + 1),0,None,LpInteger) for i in range(len(duale))]
        self.prob+=lpSum([duale[i]*x for i,x in enumerate(self.lista_var)]) #costruzione del duale
        self.prob+=lpSum([itemLengths[i]*x for i,x in enumerate(self.lista_var)])<=maxValue
        
        self.prob.writeLP('slave.lp')
        self.prob.solve()

    def risolviSlave(self):
        pattern=False
        # TEST OTTIMALITA'
        if value(self.prob.objective) > 1:
            pattern=[]
            for v in self.lista_var:
                pattern.append(value(v))
        return pattern

# Risoluzione di un'istanza

In [20]:
# GENERAZIONE ISTANZA
lunghezza_max_asse=0
lunghezza_assi=[]
domanda_assi=[]

# test istanza piccola
debug = False
mostraRisultati = False

if(debug):
	lunghezza_max_asse = 110
	lunghezza_assi=[20, 45, 50, 55, 75]
	domanda_assi=[48, 35, 24, 10, 8]
else:
	with open('instance.csv', newline='') as csvfile:
		reader = csv.reader(csvfile, delimiter=',')
		for i,row in enumerate(reader):
			if i == 0:
				lunghezza_max_asse = int(row[0])
			else:
				lunghezza_assi.append(int(row[0]))
				domanda_assi.append(int(row[1]))

if len(lunghezza_assi) != len(domanda_assi):
	print('ERRORE', "la cardinalità dell'insieme lunghezza e domanda non è uguale")
	assert False
else:
	nr_assi = len(lunghezza_assi)
	
print("Lunghezze assi : %s" % lunghezza_assi)
print("Domande assi : %s\n" % domanda_assi)

# soluzione di partenza: usare patterns con un solo asse per ogni i-esimo tipo
patterns=[]
for i in range(nr_assi):
	pattern=np.zeros(nr_assi)
	pattern[i] = 1.0
	patterns.append(pattern)

masterProblem=MasterProblem(lunghezza_max_asse,lunghezza_assi,domanda_assi,patterns,mostraRisultati)
	
solOttima=False
iterazione = 0
while solOttima==False:   # once no more new columns can be generated set relaxed to False

	duale=masterProblem.risolvi()
	slaveProblem=SlaveProblem(duale,lunghezza_assi,lunghezza_max_asse)
	newPattern=slaveProblem.risolviSlave()
	
	if newPattern:
		masterProblem.aggiungiPattern(newPattern)
		if mostraRisultati:
			print("Nuovo pattern aggiunto", newPattern)
	else:
		masterProblem.arrotonda()
		masterProblem.risolvi()
		solOttima=True
	iterazione=iterazione+1

ottimo = masterProblem.getOttimo()
print("Ottimo: %s assi da tagliare" % ottimo)
print("Trovato in %s iterazioni" % iterazione)

if mostraRisultati:
	for pattern in masterProblem.getPatternsSoluzione():
		print("%s utilizzato %s volte " % (pattern[1],pattern[0]))

Lunghezze assi : [280, 279, 277, 276, 274, 273, 272, 271, 270, 269, 268, 267, 266, 265, 264, 263, 262, 261, 260, 259, 258, 257, 256, 255, 254, 253, 252, 251, 250, 249, 248, 247, 246, 245, 243, 242, 241, 240, 239, 238, 237, 235, 234, 233, 232, 230, 228, 227, 226, 225, 224, 223, 222, 221, 220, 219, 217, 216, 215, 214, 212, 211, 210, 209, 208, 207, 206, 204, 203, 202, 201, 200, 198, 197, 195, 194, 193, 192, 191, 190, 189, 188, 187, 186, 185, 184, 183, 182, 181, 180, 179, 178, 176, 175, 174, 173, 172, 171, 170, 169, 168, 167, 165, 164, 163, 162, 161, 160, 159, 158, 157, 156, 155, 154, 152, 151, 150, 149, 148, 147, 146, 145, 143, 142, 141, 140, 139, 138, 137, 136, 135, 134, 133, 132, 131, 130, 129, 128, 127, 126, 125, 124, 123, 122, 121, 120, 119, 118, 117, 115, 114, 113, 110, 109, 108, 107, 104, 102, 101, 100, 99, 98, 97, 96, 95, 94, 92, 91, 89, 88, 87, 85, 84, 83, 82, 81, 80]
Domande assi : [2, 4, 2, 1, 2, 2, 1, 7, 1, 1, 2, 5, 4, 5, 2, 2, 4, 4, 4, 2, 1, 2, 2, 3, 5, 1, 2, 4, 2, 2, 1, 6, 5,